In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

In [2]:
#Importing data from our main page

guide_main_page_raw = 'https://steamcommunity.com/id/Zigzagzigal/myworkshopfiles/?section=guides&appid=289070'
guide_main_page = requests.get(guide_main_page_raw)
main_soup = BeautifulSoup(guide_main_page.content, "html.parser")

In [3]:
#Finding the total number of pages so we can iterate in all of them

pages = main_soup.find_all('a', attrs = {'class' : 'pagelink'})

max_pages =[]
for page in pages:
    max_pages.append(int(page.text.strip()))
max_pages = max(max_pages)

In [4]:
links = []

#We need to iterate through every page to access each guide

for i in range(1, max_pages +1):
    guide_full_page = 'https://steamcommunity.com/id/Zigzagzigal/myworkshopfiles/?section=guides&appid=289070&p=' + str(i)
    guide_page = requests.get(guide_full_page)
    guide_page_parsed = BeautifulSoup(guide_page.content, "html.parser")
    
    #We only need the left content of the page
    left_guide_page_parsed = guide_page_parsed.find(id="leftContents")
    
    #The class containing the links for our guides
    guides = left_guide_page_parsed.find_all('a', attrs = {'class' : 'workshopItemCollection ugc_show_warning_image ugc'})

    #Now we're appending to our list each individual link for each guide
    for j in range(len(guides)):
        links.append(guides[j].get('href'))
    
    
    
    

In [66]:
# Creating empty dataframe
df = pd.DataFrame(columns=['Leader','Culture', 'Diplomacy', 'Domination', 'Religion', 'Science', 'Link', 'Country'])

In [67]:
for link in range(len(links)):        
    
    
    #Iterating through every guide

    guide = requests.get(links[link])
    guide_soup = BeautifulSoup(guide.content, "html.parser")


    # Accessing the Victory Skew table
    subsections = guide_soup.find_all('div', attrs = {'subSectionTitle'})
    index = 0

    for k in range (len(subsections)):
        if subsections[k].text.strip() == 'Victory Skew':
            index = k
            break


    victory_skew = guide_soup.find_all('div', attrs = {'class' : 'subSection detailBox'})[index]
    table = victory_skew.find('div', attrs = {'class' : 'bb_table'})

    
    rows = []
    try:
        data = table.text.strip().split()

    except:
        data = []

    if ('Diplomacy' in data) or ('Diplomatic' in data) :

        data.remove('Leader')
        data.remove('Culture')        
        try:
            data.remove('Diplomacy')
        except:
            data.remove('Diplomatic')
        data.remove('Domination')
        data.remove('Religion')
        data.remove('Science')
        
        if len(data) == 6:
            rows.append(data)

        while len(data) != 6:

            while not data[1][0].isnumeric():
                leader = data[0] + ' ' + data[1]
                data.pop(0)
                data.pop(0)
                data.insert(0, leader)

            if len(data)== 6:
                rows.append(data)

            else:
                rows.append(data[0:6])
                del data[0:6]





    elif 'Leader' in data:

        data.remove('Leader')
        data.remove('Culture')
        data.remove('Domination')
        data.remove('Religion')
        data.remove('Science')

        while len(data) != 6:
            
            while not data[1][0].isnumeric():
                leader = data[0] + ' ' + data[1]
                data.pop(0)
                data.pop(0)
                data.insert(0, leader)
                
            data.insert(2, 'NaN')

            if len(data)== 6:
                rows.append(data)

            else:
                rows.append(data[0:6])
                del data[0:6]
                
    for o in range(len(rows)):
        rows[o].append(links[link])
        
    for p in range(len(rows)):
        rows[p].append(guide_soup.find('title').text.split('Zigzagzigal\'s Guides - ')[1])
                
    for m in range(len(rows)):
        df.loc[len(df)] = rows[m]


df = df[['Leader', 'Country' ,'Culture', 'Diplomacy', 'Domination', 'Religion', 'Science', 'Link']]

In [106]:
# Columns for which you want to keep only the first character
columns_to_modify = ['Culture', 'Diplomacy', 'Domination', 'Religion', 'Science']

# Applying the transformations
df['Diplomacy'] = df['Diplomacy'].apply(lambda x: '0/10' if x == 'NaN' else x)

for column in columns_to_modify:
    df[column] = df[column].apply(lambda x: int(x[0]) if isinstance(x, str) and len(x) > 0 and  x != 'NaN' and x[0:2] != '10' else x)
    df[column] = df[column].apply(lambda x : int(x[0:2]) if isinstance(x, str) and len(x) > 0 and  x != 'NaN' and x[0:2] == '10' else x)



In [135]:
# Creating a column 'Flexibility' with the average strenght of all columns (and dealing with the diplomacy column)

df['Diplomacy'] = df['Diplomacy'].replace(0, np.nan)
df['Flexibility'] = df[columns_to_modify].mean(axis=1, skipna=True)
df['Diplomacy'] = df['Diplomacy'].replace(np.nan, 0)
df['Diplomacy'] = df['Diplomacy'].apply(lambda x : int(x))

df = df[['Leader', 'Country' ,'Culture', 'Diplomacy', 'Domination', 'Religion', 'Science','Flexibility', 'Link']]

In [136]:
df

,Leader,Country,Culture,Diplomacy,Domination,Religion,Science,Flexibility,Link
0,Bà Triệu,Vietnam (GS),9,4,9,7,7,7.20,https://steamcommunity.com/sharedfiles/filedet...
1,Menelik II,Ethiopia (GS),9,4,7,10,8,7.60,https://steamcommunity.com/sharedfiles/filedet...
2,John Curtin,Australia (GS),7,8,9,7,10,8.20,https://steamcommunity.com/sharedfiles/filedet...
3,Frederick Barbarossa,Germany (GS),7,4,8,4,8,6.20,https://steamcommunity.com/sharedfiles/filedet...
4,Mvemba a Nzinga,Kongo (GS),10,7,6,0,5,5.60,https://steamcommunity.com/sharedfiles/filedet...
...,...,...,...,...,...,...,...,...,...
107,Montezuma,Aztecs (Vanilla),4,0,10,7,6,6.75,https://steamcommunity.com/sharedfiles/filedet...
108,Peter,Russia (Vanilla),8,0,7,10,4,7.25,https://steamcommunity.com/sharedfiles/filedet...
109,Saladin,Arabia (Vanilla),4,0,6,9,9,7.00,https://steamcommunity.com/sharedfiles/filedet...
110,Gorgo,Greece (Vanilla),7,0,8,4,5,6.00,https://steamcommunity.com/sharedfiles/filedet...


In [137]:
df_to_save = df.copy()
df_to_save.set_index('Leader', inplace=True)
#df_to_save.to_csv('guides.csv')